In [0]:
#code to mount google drive to read the data.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, cohen_kappa_score

#Baseline Model with Basic Feature Engineering (FE1)

In [0]:
#replace path to the path where the FE1's feather file is 
base_train = pd.read_feather('/content/drive/My Drive/introml/fe/fe1/fe1_train.feather')
base_test = pd.read_feather('/content/drive/My Drive/introml/fe/fe1/fe1_test.feather')

In [0]:
base_train = base_train.drop(['index'], axis=1)

In [0]:
base_train.shape

(17690, 37)

In [0]:
#encode the installation_id
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(base_train['installation_id'])

LabelEncoder()

In [0]:
base_train['installation_id'] = le.transform(base_train['installation_id'])

In [0]:
base_train = base_train.fillna(0)

In [0]:
x_all = base_train.drop(columns='accuracy_group')
y_all = base_train['accuracy_group']

x_train, x_test,y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

In [0]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=4, random_state=0, verbose=2)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
print('cohen kappa score:', cohen_kappa_score(y_test, y_pred))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    9.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.82      0.26      0.40       858
           1       0.95      0.04      0.07       490
           2       0.93      0.06      0.12       443
           3       0.54      0.99      0.70      1747

    accuracy                           0.57      3538
   macro avg       0.81      0.34      0.32      3538
weighted avg       0.71      0.57      0.47      3538

cohen kappa score: 0.18584089709174823


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.2s finished


#Model with contextual feature engineering FE2

In [0]:
base_train = pd.read_feather('/content/drive/My Drive/introml/fe/fe2/fe2_train.feather')
base_test = pd.read_feather('/content/drive/My Drive/introml/fe/fe2/fe2_test.feather')

In [0]:
#encode the installation_id
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(base_train['installation_id'])
base_train['installation_id'] = le.transform(base_train['installation_id']).astype(int)

In [0]:
base_train = base_train.fillna(0)
base_train['Cauldron Filler (Assessment)_4020_accuracy'] = np.log(base_train['Cauldron Filler (Assessment)_4020_accuracy'])
base_train['Mushroom Sorter (Assessment)_4020_accuracy'] = np.log(base_train['Mushroom Sorter (Assessment)_4020_accuracy'])
base_train['Bird Measurer (Assessment)_4020_accuracy'] = np.log(base_train['Bird Measurer (Assessment)_4020_accuracy'])
base_train['Chest Sorter (Assessment)_4020_accuracy'] = np.log(base_train['Chest Sorter (Assessment)_4020_accuracy'])



/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [0]:
x_all = base_train.drop(columns='accuracy_group')
y_all = base_train['accuracy_group']

x_train, x_test,y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

In [0]:
x_train = x_train.replace([np.inf, -np.inf], 0)
x_test = x_test.replace([np.inf, -np.inf], 0)


In [0]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=4, random_state=0, verbose=2)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
print('cohen kappa score:', cohen_kappa_score(y_test, y_pred))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   13.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.74      0.25      0.38       826
           1       0.00      0.00      0.00       509
           2       0.00      0.00      0.00       437
           3       0.53      0.98      0.69      1766

    accuracy                           0.55      3538
   macro avg       0.32      0.31      0.27      3538
weighted avg       0.44      0.55      0.43      3538

cohen kappa score: 0.1372364423250021


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#After Additional Feature Engineering FE3

In [0]:
train = pickle.load(open('/content/drive/My Drive/introml/fe/fe3/fe3_train.pkl', 'rb'))
test = pickle.load(open('/content/drive/My Drive/introml/fe/fe3/fe3_test.pkl', 'rb'))

In [0]:
test.shape

(1000, 517)

In [0]:
columns_w_timestamp = ['last_assessment_timestamp', 'last_game_timestamp','last_activity_timestamp']
train = train.drop(columns_w_timestamp, axis=1)
train = train.fillna(0)

# test = test.drop(columns_w_timestamp, axis=1)
# test = test.fillna(0)
x_all = train.drop(columns='accuracy_group')
y_all = train['accuracy_group']

x_train, x_test,y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

#Random Forest Classifier

##helper functions

In [0]:
def baseline_RF_Model(train_x_all, train_y_all, test_x_all, test_y_all):
  print('training model')
  clf = RandomForestClassifier(n_estimators=1000, max_depth=4, random_state=0, verbose=2)
  clf.fit(train_x_all, train_y_all)
  print('model trained')
  print(clf.score(test_x_all, test_y_all))
  return clf

def print_classification_report(model, test_x_all, test_y_all):
  y_pred = model.predict(test_x_all)
  print(classification_report(test_y_all, y_pred))
  print('cohen kappa score:', cohen_kappa_score(test_y_all, y_pred))


##Baseline Random Forest

In [0]:
base_model = baseline_RF_Model(x_train, y_train, x_test, y_test)

training model


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   21.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


0.6099491237987563


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [0]:
print_classification_report(base_model, x_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.91      0.46      0.61       858
           1       1.00      0.04      0.08       490
           2       0.00      0.00      0.00       443
           3       0.56      1.00      0.72      1747

    accuracy                           0.61      3538
   macro avg       0.62      0.37      0.35      3538
weighted avg       0.64      0.61      0.52      3538

cohen kappa score: 0.27625889512095403


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Removal of Correlated features

---



In [0]:
def remove_correlated_features(reduce_train, features):
    counter = 0
    to_remove = []
    for feat_a in features:
        for feat_b in features:
            if feat_a != feat_b and feat_a not in to_remove and feat_b not in to_remove:
                c = np.corrcoef(reduce_train[feat_a], reduce_train[feat_b])[0][1]
                if c > 0.9999:
                    counter += 1
                    to_remove.append(feat_b)
                    print('{}: FEAT_A: {} FEAT_B: {} - Correlation: {}'.format(counter, feat_a, feat_b, c))
    return to_remove

In [0]:
feat_labels = list(train.columns)
remove_list = remove_correlated_features(train, feat_labels )

1: FEAT_A: b2dba42b FEAT_B: 1bb5fbdb - Correlation: 0.9999405639639954
2: FEAT_A: dcaede90 FEAT_B: 2040 - Correlation: 1.0
3: FEAT_A: f71c4741 FEAT_B: f7e47413 - Correlation: 0.9999464999851319
4: FEAT_A: 2b9272f4 FEAT_B: 37c53127 - Correlation: 0.9999777600181912
5: FEAT_A: 2b9272f4 FEAT_B: 2050 - Correlation: 0.9999777600181912
6: FEAT_A: d88e8f25 FEAT_B: ac92046e - Correlation: 0.9999716958360618
7: FEAT_A: c277e121 FEAT_B: b120f2ac - Correlation: 0.9999845666155784
8: FEAT_A: 3bfd1a65 FEAT_B: db02c830 - Correlation: 0.9999982079383459
9: FEAT_A: 9d29771f FEAT_B: c74f40cd - Correlation: 0.9999444921075749
10: FEAT_A: 83c6c409 FEAT_B: 3dfd4aa4 - Correlation: 0.9999827146479853
11: FEAT_A: f28c589a FEAT_B: a1bbe385 - Correlation: 0.9999559134451931
12: FEAT_A: 5154fc30 FEAT_B: 3babcb9b - Correlation: 0.9999824088485433
13: FEAT_A: 0a08139c FEAT_B: 71fe8f75 - Correlation: 0.9999762369223734
14: FEAT_A: a6d66e51 FEAT_B: 5000 - Correlation: 0.9999999999999998
15: FEAT_A: 71e712d8 FEAT_B:

In [0]:
train_remove_corr = train.drop(remove_list, axis=1)
test_remove_corr = test.drop(remove_list, axis=1)

x_all = train_remove_corr.drop(columns='accuracy_group')
y_all = train_remove_corr['accuracy_group']

x_train, x_test,y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

In [0]:
feat_labels = train_remove_corr.columns

In [0]:
train_remove_corr.head()

,Clip,Activity,Assessment,Game,session_title,Magma Peak - Level 1,Sandcastle Builder (Activity),Slop Problem,Scrub-A-Dub,Tree Top City - Level 1,Ordering Spheres,All Star Sorting,Costume Box,Fireworks (Activity),12 Monkeys,Tree Top City - Level 2,Flower Waterer (Activity),Pirate's Tale,Mushroom Sorter (Assessment),Air Show,Treasure Map,Tree Top City - Level 3,Crystals Rule,Rulers,Bug Measurer (Activity),Bird Measurer (Assessment),Watering Hole (Activity),Magma Peak - Level 2,Dino Drink,Bubble Bath,Bottle Filler (Activity),Dino Dive,Crystal Caves - Level 1,Chow Time,Cauldron Filler (Assessment),Balancing Act,Crystal Caves - Level 2,Crystal Caves - Level 3,Chicken Balancer (Activity),Lifting Heavy Things,Pan Balance,Honey Cake,Happy Camel,Cart Balancer (Assessment),"Heavy, Heavier, Heaviest",Egg Dropper (Activity),Chest Sorter (Assessment),Leaf Leader,0,1,2,3,27253bdc,77261ab5,b2dba42b,1325467d,5e812b27,9ee1c98c,84538528,37937459,6d90d394,7040c096,5a848010,c1cac9a2,26fd2d99,dcaede90,f71c4741,5c3d2b2f,08fd73f3,73757a5e,2b9272f4,d88e8f25,cf82af56,4a09ace1,b7dc8128,4b5efe37,2c4e6db0,363d3849,587b5989,9e4c8c7b,1cc7cfca,2dc29e21,6043a2b4,c277e121,d45ed6a1,ca11f653,daac11b0,1f19558b,d02b7a8e,4901243f,beb0a7b9,02a42007,e694a35b,b88f38da,884228c8,9b01374f,56cd3b43,a44b10dc,bbfe0445,5d042115,de26c3a6,598f4598,fcfdffb6,3bfd1a65,a1e4395d,a52b92d5,28ed704e,9d29771f,7da34a02,83c6c409,fbaf3456,5f0eb72c,c7128948,25fa8af4,6c930e6e,a5be6304,15ba1109,65abac75,bcceccc6,06372577,f28c589a,d88ca108,dcb55a27,1575e76c,28f975ea,14de4c5d,9b4001e4,58a0de5c,f5b8c21a,7423acbc,e04fb33d,48349b14,cc5087a3,5154fc30,7cf1bc53,5e3ea25a,86c924c4,3ddc79c3,e720d930,3323d7e9,44cb4907,8b757ab8,c7f7f0e1,0a08139c,e79f3763,363c86c9,022b4259,565a3990,f56e0afc,ec138c1c,1375ccb7,bdf49a58,51102b85,4a4c3d21,17113b36,ad2fc29c,e37a2b78,a16a373e,c58186bf,e64e2cfd,49ed92e9,bd701df8,f50fc6c1,d2e9262e,2fb91ec1,c952eb01,a6d66e51,71e712d8,51311d7a,5be391b5,c6971acf,a29c5338,6c517a88,7f0836bf,4d6737eb,f806dc10,792530f8,1996c610,74e5f8a7,e5734469,89aace00,77ead60d,4d911100,16dffff1,1cf54632,8d84fa81,99abe2bb,99ea62f3,0413e89d,15eb4a7d,1beb320a,8f094001,a0faea5d,c54cf6c5,857f21c0,6f4adc4b,55115cbd,90ea0bac,5859dfb6,d06f75b5,895865f3,ecc36b7f,1340b8d7,d2278a3b,b7530680,67439901,bb3e370b,df4940d3,90efca10,d3f1e122,e9c52111,15a43e5b,160654fd,88d4a5be,d2659ab4,45d01abe,7525289a,f6947f54,8fee50e2,070a5291,3393b68b,9ed8f6da,ad148f58,29bdd9ba,87d743c1,832735e1,76babcde,ab3136ba,7d5c30a2,7d093bf9,f93fc684,7ec0c298,0d1da71f,63f13dd7,7372e1a5,cfbd47c8,d185d3ea,4ef8cdd3,0330ab6a,2230fab4,56817e2b,47026d5f,cb6010f8,e3ff61fb,709b1251,6088b756,7961e599,c0415e5c,9de5e594,28a4eb9a,00c73085,90d848e0,532a2afb,923afab1,3ee399c3,37ee8496,30614231,2dcad279,392e14df,28520915,d3268efa,b5053438,756e5507,ea321fb1,84b0e0c8,4bb2f698,56bcd38d,499edb7c,cdd22e43,46cd75b4,85d1b0de,9c5ef70c,0086365d,a592d54e,6cf7d25c,a76029ee,d38c2fd7,731c0cbe,8d748b58,47efca07,5f5b2617,0d18d96c,d9c005dd,abc5811c,d51b1749,3bb91ced,c2baf0bd,8d7e386c,3d8c61b0,6bf9e3e1,69fdac0a,8af75982,1af8be29,3bf1cf26,a7640a16,36fa3ebe,c7fe2a55,a8a78786,7ad3efc6,795e4a37,5e109ec3,5c2f29ca,828e68f9,d122731b,a8876db3,ecaab346,b2e5b0f1,f3cd5473,15f99afc,2a444e03,804ee27f,a5e9da97,907a054b,e7561dd2,bc8f2793,9b23e8ee,7ab78247,b80e5e84,4c2ec19f,5b49460a,3dcdda7f,3ccd3f02,bd612267,a8efe47b,562cec5f,3d0b9317,0db6d71d,9ce586dd,93b353f2,ea296733,df4fe8b6,f54238ee,9e6b7fb5,b1d5101d,a1192f43,47f43a44,6f4bd64e,c189aaf2,37db1c2f,461eace6,9e34ea74,8ac7cce4,fd20ea40,29f54413,33505eae,7dfe6d8a,2a512369,86ba578b,f32856e4,262136f4,b012cd7f,e5c9df6f,763fc34e,e57dd7af,67aa2ada,cb1178ad,e7e44842,46b50ba8,a2df0760,6aeafed4,e080a381,1c178d24,cf7638f3,9d4e7b25,acf5c23f,91561152,2b058fe3,3d63345e,53c6e11a,92687c59,5de79a6a,31973d56,4e5fc6f5,6f8106d9,04df9b66,5290eab1,3edf6747,3b2048ee,5348fd84,3afb49e6,e4f1efe6,38074c54,19967db1,05ad839b,eb2c19cd,08ff79ad,611485c5,93edfe2e,d3640339,77c76bc5,29a42aea,6f445b57,26a5a3dd,13f56524,3a4be871,2ec694de,dcb1663e,30df3273,bfc77bd6,16667cc

###Base line after removal of correlated features

In [0]:
base_model_removed_corr = baseline_RF_Model(x_train, y_train, x_test, y_test)

training model


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   20.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


0.6116449971735444


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [0]:
print_classification_report(base_model_removed_corr, x_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.91      0.46      0.61       858
           1       1.00      0.06      0.11       490
           2       0.00      0.00      0.00       443
           3       0.57      1.00      0.72      1747

    accuracy                           0.61      3538
   macro avg       0.62      0.38      0.36      3538
weighted avg       0.64      0.61      0.52      3538

cohen kappa score: 0.2803442472168036


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Feature Selection

In [0]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.001
sfm = SelectFromModel(base_model_removed_corr, threshold=0.001)

# Train the selector
sfm.fit(x_all, y_all)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   25.7s finished


SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini', max_depth=4,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=1000, n_jobs=None,
                                                 oob_score=False,
                                                 r

In [0]:
selected_features = []
for feature_list_index in sfm.get_support(indices=True):
    selected_features.append(feat_labels[feature_list_index])

In [0]:
selected_features

['Clip',
 'Assessment',
 'session_title',
 'Mushroom Sorter (Assessment)',
 'Bird Measurer (Assessment)',
 'Cauldron Filler (Assessment)',
 'Cart Balancer (Assessment)',
 'Chest Sorter (Assessment)',
 0,
 1,
 2,
 3,
 '3bfd1a65',
 'a52b92d5',
 '9d29771f',
 '7da34a02',
 '83c6c409',
 'fbaf3456',
 '5f0eb72c',
 'c7128948',
 '25fa8af4',
 '6c930e6e',
 'a5be6304',
 'ec138c1c',
 '1375ccb7',
 'bdf49a58',
 '51102b85',
 '4a4c3d21',
 '17113b36',
 'ad2fc29c',
 'e37a2b78',
 'a16a373e',
 '160654fd',
 '88d4a5be',
 '45d01abe',
 '7525289a',
 'f6947f54',
 '8fee50e2',
 '070a5291',
 '3393b68b',
 '532a2afb',
 '923afab1',
 '3ee399c3',
 '37ee8496',
 '30614231',
 '2dcad279',
 '392e14df',
 '28520915',
 'd3268efa',
 'b5053438',
 '7ad3efc6',
 '795e4a37',
 '5e109ec3',
 '5c2f29ca',
 '828e68f9',
 'd122731b',
 'a8876db3',
 'ecaab346',
 'b2e5b0f1',
 '5b49460a',
 '3dcdda7f',
 '3ccd3f02',
 'bd612267',
 'a8efe47b',
 '562cec5f',
 '0db6d71d',
 '9ce586dd',
 '93b353f2',
 'ea296733',
 'df4fe8b6',
 'acf5c23f',
 '91561152',
 '2b

In [0]:
len(selected_features)

105

In [0]:
selected_features.append('installation_id')

In [0]:
filtered_train = train_remove_corr[selected_features]
filtered_test = test_remove_corr[selected_features]
feature_labels = filtered_train.columns



In [0]:
filtered_test.head()

,Clip,Assessment,session_title,Mushroom Sorter (Assessment),Bird Measurer (Assessment),Cauldron Filler (Assessment),Cart Balancer (Assessment),Chest Sorter (Assessment),0,1,2,3,3bfd1a65,a52b92d5,9d29771f,7da34a02,83c6c409,fbaf3456,5f0eb72c,c7128948,25fa8af4,6c930e6e,a5be6304,ec138c1c,1375ccb7,bdf49a58,51102b85,4a4c3d21,17113b36,ad2fc29c,e37a2b78,a16a373e,160654fd,88d4a5be,45d01abe,7525289a,f6947f54,8fee50e2,070a5291,3393b68b,532a2afb,923afab1,3ee399c3,37ee8496,30614231,2dcad279,392e14df,28520915,d3268efa,b5053438,7ad3efc6,795e4a37,5e109ec3,5c2f29ca,828e68f9,d122731b,a8876db3,ecaab346,b2e5b0f1,5b49460a,3dcdda7f,3ccd3f02,bd612267,a8efe47b,562cec5f,0db6d71d,9ce586dd,93b353f2,ea296733,df4fe8b6,acf5c23f,91561152,2b058fe3,5de79a6a,31973d56,04df9b66,5290eab1,3afb49e6,e4f1efe6,38074c54,2000,4030,4020,2030,3021,3020,3120,4025,4100,2010,4110,Cauldron Filler (Assessment)_4020_accuracy,Bird Measurer (Assessment)_4020_accuracy,correct_Mushroom Sorter (Assessment),wrong_Bird Measurer (Assessment),correct_Cart Balancer (Assessment),wrong_Cart Balancer (Assessment),correct_Chest Sorter (Assessment),correct_Cauldron Filler (Assessment),wrong_Cauldron Filler (Assessment),assessment_durations_mean,Assessment_gametime,accuracy_group,last_assessment_world,last_assessment_event_count,installation_id
0,5,2,38,0,0,1,26,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,5,4,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,17,107,90,3,3,1,1,10,1,1,0,0.0,0.0,0,0,1,0,0,0,0,0.0,0,3,0.0,1.0,2138
1,14,5,38,51,61,36,14,83,2,1,2,1,1,4,4,10,1,6,5,1,2,1,1,2,4,4,14,8,3,3,3,7,1,1,1,1,2,5,2,1,2,4,3,4,4,4,2,2,3,3,2,1,2,2,1,1,1,1,1,1,4,4,26,19,12,3,1,3,3,3,0,1,1,0,0,1,1,0,0,0,49,384,304,76,66,24,24,87,23,4,3,0.0,0.0,1,2,1,0,0,1,1,46.6,233,3,2.0,83.0,2163
2,0,1,13,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,4423
3,2,1,13,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,61,31,3,3,2,2,9,0,0,0,0.0,0.0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,602
4,1,1,38,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,3162


In [0]:
# test_x_all = filtered_test.drop(['accuracy_group'], axis=1)
# test_y_all = filtered_test['accuracy_group']

x_all = filtered_train.drop(columns='accuracy_group')
y_all = filtered_train['accuracy_group']

x_train, x_test,y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42)

In [0]:
base_model_removed_w_selected = baseline_RF_Model(x_train, y_train, x_test, y_test)

training model


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   12.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


0.6913510457885811


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [0]:
print_classification_report(base_model_removed_w_selected, x_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.90      0.57      0.70       858
           1       0.95      0.23      0.37       490
           2       0.98      0.23      0.37       443
           3       0.63      1.00      0.77      1747

    accuracy                           0.69      3538
   macro avg       0.86      0.51      0.55      3538
weighted avg       0.78      0.69      0.65      3538

cohen kappa score: 0.4563014981945419


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished


##Random grid CV search

In [0]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [0]:
clf = RandomForestClassifier(random_state=42)
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=3, random_state=42)
# Fit the random search model
rf_random.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, score=0.782, total=   5.1s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, score=0.786, total=   5.1s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.2s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, score=0.791, total=   5.1s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, score=0.784, total=  21.9s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, score=0.788, total=  22.4s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False, score=0.791, total=  21.9s
[CV] n_estimators=600, min_samples_split=2, min_samples_leaf=2, max_features=auto, m

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 15.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [0]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 600}

In [0]:
rf_cv_clf = RandomForestClassifier(n_estimators=1000, max_depth=80, random_state=0, verbose=2,min_samples_leaf=1
                                    , min_samples_split=10, max_features='auto', bootstrap=False)
rf_cv_clf.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   56.6s finished


RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=80, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=2,
                       warm_start=False)

In [0]:
y_pred = rf_cv_clf.predict(x_test)
print(classification_report(y_test, y_pred))
print('cohen kappa score:', cohen_kappa_score(y_test, y_pred))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.86      0.83      0.84       858
           1       0.75      0.58      0.66       490
           2       0.73      0.49      0.59       443
           3       0.79      0.92      0.85      1747

    accuracy                           0.79      3538
   macro avg       0.78      0.70      0.73      3538
weighted avg       0.79      0.79      0.79      3538

cohen kappa score: 0.6762809872984497


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.8s finished


In [0]:
rf_cv_clf = RandomForestClassifier(n_estimators=600, max_depth=90, random_state=0, verbose=2,min_samples_leaf=4
                                    , min_samples_split=10, max_features='sqrt', bootstrap=False)
rf_cv_clf.fit(x_train, y_train)


y_pred = rf_cv_clf.predict(x_test)
print(classification_report(y_test, y_pred))
print('cohen kappa score:', cohen_kappa_score(y_test, y_pred))
# rf_cv_clf.score(test_x_all, test_y_all) 

# {'bootstrap': False,
#  'max_depth': 90,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 4,
#  'min_samples_split': 10,
#  'n_estimators': 600}

building tree 1 of 600


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 2 of 600
building tree 3 of 600
building tree 4 of 600
building tree 5 of 600
building tree 6 of 600
building tree 7 of 600
building tree 8 of 600
building tree 9 of 600
building tree 10 of 600
building tree 11 of 600
building tree 12 of 600
building tree 13 of 600
building tree 14 of 600
building tree 15 of 600
building tree 16 of 600
building tree 17 of 600
building tree 18 of 600
building tree 19 of 600
building tree 20 of 600
building tree 21 of 600
building tree 22 of 600
building tree 23 of 600
building tree 24 of 600
building tree 25 of 600
building tree 26 of 600
building tree 27 of 600
building tree 28 of 600
building tree 29 of 600
building tree 30 of 600
building tree 31 of 600
building tree 32 of 600
building tree 33 of 600
building tree 34 of 600
building tree 35 of 600
building tree 36 of 600
building tree 37 of 600
building tree 38 of 600
building tree 39 of 600
building tree 40 of 600
building tree 41 of 600
building tree 42 of 600
building tree 43 of 600


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:   31.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


              precision    recall  f1-score   support

           0       0.86      0.81      0.84       858
           1       0.78      0.58      0.67       490
           2       0.78      0.46      0.58       443
           3       0.78      0.94      0.85      1747

    accuracy                           0.80      3538
   macro avg       0.80      0.70      0.73      3538
weighted avg       0.80      0.80      0.79      3538

cohen kappa score: 0.6787275733104479


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    0.5s finished
